##This notebook will be used for the capstone project

In [3]:
import pandas as pd

In [4]:
import numpy as np

In [6]:
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


In [ ]:
import folium
import json
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import requests

from geopy.geocoders import Nominatim
from pandas.io.json import json_normalize
from scipy.spatial.distance import cdist
from sklearn.cluster import KMeans

print('Libraries imported.')

In [ ]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    categoryID = catid 
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&categoryId={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT,
            categoryID)
            
        
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

# This function will take a row and sort its column values in descending
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]


# This function will take an arbitrary cluster number and plot a bar chart of the top ten
# food-related vunues found within the cluster.
def plot_cluster(c_num):
    c = sf_merged.loc[sf_merged['Cluster Labels'] == c_num, sf_merged.columns[[1] + list(range(5, ny_merged.shape[1]))]]
    c_nh = c[['Neighborhood']] 
    c_nh = c_nh.merge(sf_totals, on=['Neighborhood'], how='inner')

    c_nh.loc['Total'] = c_nh.sum() # Add total row at bottom of dataframe
    c_nh.at['Total', 'Neighborhood'] = 'All neighborhoods' # Change value to remove concatenation of all neighborhood names

    tot_c = c_nh.loc[['Total']]
    tot_c = tot_c.drop(columns={'Neighborhood'}) # Drop the neighborhood column
    tot_c = tot_c.loc['Total'].sort_values(ascending=False) # Sort the categories in descending order
    tot_c = tot_c[:10].sort_values(ascending=True) # Take the top 10 categories only and then sort in ascending order

    fig = go.Figure(go.Bar(
                x=tot_c.values,
                y=tot_c.index,
                orientation='h'))

    fig.update_layout(
        title="Top 10 Deli Venues in Cluster " + str(c_num),
        xaxis_title="Quantity",
    )

    fig.show()
    return

In [ ]:
with open('suffolk_data.json') as json_data:
    newyork_data = json.load(json_data)

# relevant data is in the features key    
neighborhoods_data = newyork_data['features']

# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

# iterate through the data file and load into dataframe one row at a time
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)
    
# examine dataframe to ensure data loaded properly
neighborhoods.head()

In [ ]:
sf_venues = getNearbyVenues(names=neighborhoods['Neighborhood'],
                                   latitudes=neighborhoods['Latitude'],
                                   longitudes=neighborhoods['Longitude']
                                  )